#  Movie Insights Analysis
  
**Group Members:** Mathews Wandera, Tinah, Diana, Pacificah, Night, Frank  
**Branch Owners:**  
- `Mathews-Tableau`  
- `Tinah-Presentation`  
- `Diana-Data_Preparation`  
- `Pacificah-EDA`  
- `Night-Visualization`  
- `Frank-README`  

---

### Objective
To analyze multiple movie-related datasets from various platforms (IMDb, Rotten Tomatoes, The Numbers, TMDb, and Box Office Mojo) and provide strategic insights to support decision-making in the film industry.

## 1. Business Understanding

### Problem Statement
The movie industry produces a vast number of films every year, but only a few turn out to be box office hits or critical successes. Stakeholders—such as producers, distributors, and marketing teams—require data-driven insights to inform their decisions. By analyzing trends in budgets, genres, ratings, and revenues, we aim to uncover factors that contribute to a movie's commercial and critical success.

### Project Goal
This project seeks to:
- Identify the key characteristics of successful movies.
- Understand how different platforms rate movies (IMDb, Rotten Tomatoes, etc.).
- Analyze financial patterns like budget vs. revenue.
- Provide data-driven recommendations for improving the profitability and impact of future productions.

### Key Questions
- What genres perform best in terms of revenue and ratings?
- How do production budgets relate to revenue or critical scores?
- Are there specific patterns in release dates that impact success?
- What are the most profitable platforms or combinations of features?

### Target Audience
- Film producers and investors
- Distribution and marketing teams
- Data-driven creative teams in media
- Streaming platform analysts

## 2. Data Understanding
We'll now systematically explore each dataset to understand:

Structure (columns and data types)

- Sample data (via .head())

- Missing values

- Duplicates

- Basic statistics (via .describe())

We'll do this for:

**bom.movie_gross.csv.gz**

**rt.movie_info.tsv.gz**

**rt.reviews.tsv.gz**

**tmdb.movies.csv.gz**

**tn.movie_budgets.csv.gz**

Key tables from `im.db`: movie_basics, movie_ratings, principals, etc.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import zipfile
import os

## Confirm available files

In [3]:

# Confirm available files
print(" Contents of 'datasets':")
print(os.listdir('datasets'))

 Contents of 'datasets':
['tmdb.movies.csv.gz', 'im.db.zip', 'rt.movie_info.tsv.gz', '.gitkeep', 'bom.movie_gross.csv.gz', 'rt.reviews.tsv.gz', 'im.db', '.gitignore', 'tn.movie_budgets.csv.gz']


## Load external CSV/TSV dataset

In [4]:
# Load external CSV/TSV datasets
bom = pd.read_csv("datasets/bom.movie_gross.csv.gz")
rt_info = pd.read_csv("datasets/rt.movie_info.tsv.gz", sep="\t")
rt_reviews = pd.read_csv("datasets/rt.reviews.tsv.gz", sep="\t", encoding="latin1")  # fixed encoding issue
tmdb = pd.read_csv("datasets/tmdb.movies.csv.gz")
tn_budgets = pd.read_csv("datasets/tn.movie_budgets.csv.gz")


## Unzip the im.db.zip

In [5]:
if not os.path.exists("datasets/im.db"):
    with zipfile.ZipFile("datasets/im.db.zip", 'r') as zip_ref:
        zip_ref.extractall("datasets")

In [6]:
#Connect to the SQLite database
conn = sqlite3.connect("datasets/im.db")

# Preview available tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("\n Tables in im.db:")
print(tables)


 Tables in im.db:
            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


## 3. Data Preparation

Now that we have successfully loaded all datasets and explored their structure, we'll proceed with data preparation. This involves:

- Merging key datasets into a master data table.
- Cleaning and standardizing fields.
- Saving the final cleaned dataset for EDA and Tableau use.


## Step 1 — Prepare Individual DataFrames

### 🔹 Step 1: Select and Rename Relevant Columns

We’ll simplify each dataset and rename columns for consistency before merging. This ensures our final dataset is clean and easy to work with.


In [8]:
# Check column names in Rotten Tomatoes info dataset
print(rt_info.columns.tolist())


['id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime', 'studio']


### 1: BOM (Box Office Mojo)

In [9]:
# BOM dataset (Box Office Mojo)
bom_clean = bom[['title', 'domestic_gross', 'foreign_gross', 'year']].copy()
bom_clean.rename(columns={'title': 'movie_title'}, inplace=True)


###  2.TN (The Numbers) Budgets

In [10]:
# TN Budgets dataset
tn_budgets_clean = tn_budgets[['movie', 'production_budget', 'worldwide_gross', 'release_date']].copy()
tn_budgets_clean.rename(columns={'movie': 'movie_title'}, inplace=True)


### 3: TMDb

In [11]:
# TMDb dataset
tmdb_clean = tmdb[['title', 'id', 'popularity', 'vote_average', 'vote_count']].copy()
tmdb_clean.rename(columns={'title': 'movie_title', 'id': 'tmdb_id'}, inplace=True)


### 4: Rotten Tomatoes (info)

In [12]:
# Rotten Tomatoes info dataset
rt_info_clean = rt_info[['genre', 'rating', 'studio', 'theater_date', 'runtime']].copy()

# No title column, so we cannot rename to movie_title here — will join later if possible


## Step 2 – Load Tables from the SQLite Database
We'll extract and preview the following key tables:

- movie_basics – core movie metadata

- movie_ratings – IMDb average rating + votes

- principals – cast/crew info

- optionally) names, crew, etc., if needed later



### 5: Connect to the SQLite DB and List Tables

In [13]:
# Connect to the SQLite database
conn = sqlite3.connect('datasets/im.db')

# Check available tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql(tables_query, conn)


,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


### 6: Load Key Tables into DataFrames

In [14]:
# Load movie_basics
movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)

# Load movie_ratings
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)

# Load principals (optional for later joins with crew/names)
principals = pd.read_sql("SELECT * FROM principals", conn)


### 7: Merge IMDb tables

**We merge movie_basics and movie_ratings on movie_id.**

In [16]:
# Merge basics and ratings
imdb_movies = pd.merge(movie_basics, movie_ratings, how='left', on='movie_id')

# Preview merged IMDb data
imdb_movies.head()


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


## Merging the data
Let's proceed step by step to merge the cleaned datasets into one master DataFrame that we'll use for EDA and Tableau.
**Step-by-Step Plan**
- Ensure all individual datasets are cleaned and renamed

- Merge them progressively using movie_title as the key

- Handle duplicates and missing values

- Save final master dataset to the datasets/ folder

### Step 1: Clean and Rename Columns for Merging



In [17]:
# BOM dataset: drop unnecessary columns and rename for consistency
bom_clean = bom[['title', 'domestic_gross', 'foreign_gross', 'year']].copy()
bom_clean.rename(columns={'title': 'movie_title'}, inplace=True)
bom_clean.head()


,movie_title,domestic_gross,foreign_gross,year
0,Toy Story 3,415000000.0,652000000,2010
1,Alice in Wonderland (2010),334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000,2010
3,Inception,292600000.0,535700000,2010
4,Shrek Forever After,238700000.0,513900000,2010


In [18]:
# The Numbers movie budget data
tn_budgets_clean = tn_budgets[['movie', 'production_budget', 'worldwide_gross', 'release_date']].copy()
tn_budgets_clean.rename(columns={'movie': 'movie_title'}, inplace=True)
tn_budgets_clean.head()


,movie_title,production_budget,worldwide_gross,release_date
0,Avatar,"$425,000,000","$2,776,345,279","Dec 18, 2009"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875","May 20, 2011"
2,Dark Phoenix,"$350,000,000","$149,762,350","Jun 7, 2019"
3,Avengers: Age of Ultron,"$330,600,000","$1,403,013,963","May 1, 2015"
4,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$1,316,721,747","Dec 15, 2017"


In [19]:
# TMDb: includes popularity, vote average, vote count
tmdb_clean = tmdb[['title', 'id', 'popularity', 'vote_average', 'vote_count']].copy()
tmdb_clean.rename(columns={'title': 'movie_title', 'id': 'tmdb_id'}, inplace=True)
tmdb_clean.head()


,movie_title,tmdb_id,popularity,vote_average,vote_count
0,Harry Potter and the Deathly Hallows: Part 1,12444,33.533,7.7,10788
1,How to Train Your Dragon,10191,28.734,7.7,7610
2,Iron Man 2,10138,28.515,6.8,12368
3,Toy Story,862,28.005,7.9,10174
4,Inception,27205,27.920,8.3,22186


In [20]:
# Rotten Tomatoes info data
rt_info_clean = rt_info[['rating', 'genre', 'studio', 'theater_date', 'runtime']].copy()
rt_info_clean['movie_title'] = rt_info.index
rt_info_clean.reset_index(drop=True, inplace=True)
rt_info_clean.head()


,rating,genre,studio,theater_date,runtime,movie_title
0,R,Action and Adventure|Classics|Drama,NaN,"Oct 9, 1971",104 minutes,0
1,R,Drama|Science Fiction and Fantasy,Entertainment One,"Aug 17, 2012",108 minutes,1
2,R,Drama|Musical and Performing Arts,NaN,"Sep 13, 1996",116 minutes,2
3,R,Drama|Mystery and Suspense,NaN,"Dec 9, 1994",128 minutes,3
4,NR,Drama|Romance,NaN,NaN,200 minutes,4


## Step 2: Merging All Cleaned Datasets
Let’s now proceed cell by cell to merge them using movie_title as the key.

### Cell 1: Merge BOM + The Numbers Budgets

In [21]:
# Merge BOM and TN Budgets
bom_tn_merged = pd.merge(bom_clean, tn_budgets_clean, on='movie_title', how='outer')
bom_tn_merged.head()


,movie_title,domestic_gross,foreign_gross,year,production_budget,worldwide_gross,release_date
0,Toy Story 3,415000000.0,652000000,2010.0,"$200,000,000","$1,068,879,522","Jun 18, 2010"
1,Alice in Wonderland (2010),334200000.0,691300000,2010.0,NaN,NaN,NaN
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000,2010.0,NaN,NaN,NaN
3,Inception,292600000.0,535700000,2010.0,"$160,000,000","$835,524,642","Jul 16, 2010"
4,Shrek Forever After,238700000.0,513900000,2010.0,"$165,000,000","$756,244,673","May 21, 2010"


### Cell 2: Add TMDb Data

In [22]:
# Merge with TMDb data
bom_tn_tmdb = pd.merge(bom_tn_merged, tmdb_clean, on='movie_title', how='outer')
bom_tn_tmdb.head()


,movie_title,domestic_gross,foreign_gross,year,production_budget,worldwide_gross,release_date,tmdb_id,popularity,vote_average,vote_count
0,Toy Story 3,415000000.0,652000000,2010.0,"$200,000,000","$1,068,879,522","Jun 18, 2010",10193.0,24.445,7.7,8340.0
1,Alice in Wonderland (2010),334200000.0,691300000,2010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000,2010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Inception,292600000.0,535700000,2010.0,"$160,000,000","$835,524,642","Jul 16, 2010",27205.0,27.920,8.3,22186.0
4,Shrek Forever After,238700000.0,513900000,2010.0,"$165,000,000","$756,244,673","May 21, 2010",10192.0,15.041,6.1,3843.0


### Cell 3: Add Rotten Tomatoes Info

In [24]:
# Ensure movie_title is a string before merge
rt_info_clean['movie_title'] = rt_info_clean['movie_title'].astype(str)
bom_tn_tmdb['movie_title'] = bom_tn_tmdb['movie_title'].astype(str)

# Now merge
master_df = pd.merge(bom_tn_tmdb, rt_info_clean, on='movie_title', how='outer')
master_df.head()


,movie_title,domestic_gross,foreign_gross,year,production_budget,worldwide_gross,release_date,tmdb_id,popularity,vote_average,vote_count,rating,genre,studio,theater_date,runtime
0,Toy Story 3,415000000.0,652000000,2010.0,"$200,000,000","$1,068,879,522","Jun 18, 2010",10193.0,24.445,7.7,8340.0,NaN,NaN,NaN,NaN,NaN
1,Alice in Wonderland (2010),334200000.0,691300000,2010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000,2010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Inception,292600000.0,535700000,2010.0,"$160,000,000","$835,524,642","Jul 16, 2010",27205.0,27.920,8.3,22186.0,NaN,NaN,NaN,NaN,NaN
4,Shrek Forever After,238700000.0,513900000,2010.0,"$165,000,000","$756,244,673","May 21, 2010",10192.0,15.041,6.1,3843.0,NaN,NaN,NaN,NaN,NaN
